# Exploratory Data Analysis

#### We load in our data, reshape and merge

In [ ]:
import pandas as pd

basics = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.basics.tsv.gz", sep = "\t", compression = "gzip", na_values="\\N")
ratings = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.ratings.tsv.gz", sep = "\t", compression = "gzip")

C:\Users\jungn\AppData\Local\Temp\ipykernel_15308\1574588733.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.basics.tsv.gz", sep = "\t", compression = "gzip", na_values="\\N")


#### from the IMDB dataset we only want movies (between 1980-2022), so we exclude tv shows, videos etc

In [ ]:
movies = basics[(basics["titleType"] == "movie") & (basics["startYear"].between(1980, 2022))]
[["tconst", "primaryTitle", "startYear", "runtimeMinutes", "genres"]]

#### We now merge the movies data and ratings data based on "tconst" which is the unique movie ID

In [ ]:
movies = movies.merge(ratings, on="tconst", how="left")

#### Load in the principals data 

In [ ]:
principals = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.principals.tsv.gz", sep="\t", na_values="\\N")

#### We get the director count and cast size

In [ ]:
principals = principals[principals["category"].isin(["director", "actor", "actress"])]

In [ ]:
director_counts = (principals[principals["category"] == "director"]
    .groupby("tconst")
    .size()
    .rename("num_directors")
    .reset_index()
)

In [ ]:
cast_size = (principals[principals["category"].isin(["actor", "actress"])]
    .groupby("tconst")
    .size()
    .rename("cast_size")
    .reset_index()
)

In [ ]:
movies = (movies
    .merge(director_counts, on="tconst", how="left")
    .merge(cast_size, on="tconst", how="left")
)


In [ ]:
movies[["num_directors", "cast_size"]] = (movies[["num_directors", "cast_size"]].fillna(0))


#### Convert into a csv file 

In [ ]:
movies.to_csv("C:/Users/jungn/OneDrive/Documents/movie project/movies_features.csv", index=False)


In [ ]:
films = pd.read_csv("C:/Users/jungn/OneDrive/Documents/movie project/movies_features.csv")

In [ ]:
films

,tconst,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,num_directors,cast_size
0,tt0011801,Tötet nicht mehr,2019.0,NaN,"Action,Crime",NaN,NaN,1.0,10.0
1,tt0015724,Dama de noche,1993.0,102.0,"Drama,Mystery,Romance",6.2,35.0,1.0,10.0
2,tt0035423,Kate & Leopold,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,92914.0,1.0,10.0
3,tt0036606,"Another Time, Another Place",1983.0,118.0,"Drama,War",6.4,379.0,1.0,10.0
4,tt0038086,Shiva und die Galgenblume,1993.0,NaN,Thriller,7.0,30.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...
395550,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015.0,57.0,Documentary,NaN,NaN,1.0,1.0
395551,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007.0,100.0,Documentary,NaN,NaN,1.0,0.0
395552,tt9916706,Dankyavar Danka,2013.0,NaN,Comedy,7.7,9.0,1.0,10.0
395553,tt9916730,6 Gunn,2017.0,116.0,Drama,7.0,13.0,1.0,8.0


#### Now we move on to some exploring

In [ ]:
# Total NaN count:
films.isna().any(axis=1).sum()


np.int64(194669)

In [ ]:
# Which columns have NaNs and how many?
films.isna().sum().sort_values(ascending=False)


numVotes          168946
averageRating     168946
runtimeMinutes     89259
genres             30256
primaryTitle           2
tconst                 0
startYear              0
num_directors          0
cast_size              0
dtype: int64

Above, we see that numVotes and averageRating have the same number of NaNs which means they are missing together. Movies without an IMDB rating is an indication that they are very obscure, and since this project is about how a movie ages in audience pereption these will not serve any purpose so they can be dropped.

In [ ]:
films = films.dropna(subset = ["averageRating", "numVotes"])
films = films.dropna(subset = ["primaryTitle"])
# drop the 2 primaryTitle NaNs as it is a negligible amount

I have decided to keep the rest (genres, runtimeMinutes) as they are still important and will prevent potential bias towards mainstream cinema.

#### Handling runtime NaNs:

In [ ]:
films["runtime_missing"] = films["runtimeMinutes"].isna().astype(int)
films["runtimeMinutes"] = films["runtimeMinutes"].fillna(films["runtimeMinutes"].median())
# Create a new column indicating which rows were NaNs as an integer (0,1) 
# Replace missing values with median runtime of the dataset (mean is not as robust when it comes to extreme values or outliers)